In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder
import re

In [62]:
df = pd.read_csv(r"C:\Users\Romar\Google Drive\Master\DE\DE-group-5\Assignment 2\Data\data.csv")
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [63]:
df.columns

Index(['Unnamed: 0', 'ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag',
       'Overall', 'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW',
       'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingT

In [64]:
def PreprocessData(df):
    #Remove all NaN rows
    df_preprocessed = df.dropna()
    
    #one hot encoding nationality
    encoder = TargetEncoder()
    df_preprocessed['Nationality Encoded'] = encoder.fit_transform(df_preprocessed['Nationality'], df_preprocessed['Overall'])


    #one hot encoding club (remove NaN first)
    encoder1 = TargetEncoder()
    df_preprocessed['Club Encoded'] = encoder1.fit_transform(df_preprocessed['Club'], df_preprocessed['Overall'])
    #onehot encoding preferred foot (remove NaN first)
    #encode Workrate
    #encode BodyType
    #encode position
    #Value
    #Wage
    wage = df_preprocessed.Wage
    newwage = []
    for ITEM in wage:
        temp = ITEM.split("€")[1]
        newwage.append(float(temp.split("K")[0]))
    df_preprocessed['Wage'] = newwage
    newwage = []
    
    #Weight
    weight = df_preprocessed.Weight
    newweight = []
    for ITEM  in weight:
        newweight.append(float(ITEM.split("lbs")[0]))
    df_preprocessed['Weight'] = newweight
    newweight = []

    #Height
    newheight = []
    height = df_preprocessed.Height
    for ITEM in height:
        feet, inch = float(ITEM.split("'")[0]), float(ITEM.split("'")[1])
        newheight.append(feet*30.48 + inch*2.54)
    df_preprocessed['Height'] = newheight
    newheight = []
    #All the positions, weet niet wat we daar mee willlen
    positions = ['LS','ST' , 'RS', 'LW', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM','RCM', 'RM','LWB',
                 'LDM','CDM', 'RDM','RWB', 'LB', 'LCB', 'CB', 'RCB','RB']

    for position in positions:
        print(position)
        data = []
        new_data = []
        data = df_preprocessed[position]
        for ITEM in data:
            new_data.append(float(ITEM.split("+")[0]) + float(ITEM.split("+")[1]))
        df_preprocessed[position] = new_data
        new_data = []
    return df_preprocessed
def SplitData(df):
    df.pop('ID')
    df.pop('Unnamed: 0')
    df.pop('Photo')
    df.pop('Flag')
    df.pop('Club Logo')
    df.pop('Release Clause')
    df.pop('Name')
    df.pop('Loaned From')
    df.pop('Real Face')
    df.pop('Contract Valid Until')

    df_pp = PreprocessData(df)

    y = df_pp.pop('Overall')
    X_train, X_test, y_train, y_test = train_test_split(df_pp, y, test_size=0.33, random_state=42)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = SplitData(df)

LS
ST
RS
LW
CF
RF
RW
LAM
CAM
RAM
LM
LCM
CM
RCM
RM
LWB
LDM
CDM
RDM
RWB
LB
LCB
CB
RCB
RB


C:\ProgramData\Anaconda3\envs\Assignment 2\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
<ipython-input-64-caa0ba1998fd>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_preprocessed['Nationality Encoded'] = encoder.fit_transform(df_preprocessed['Nationality'], df_preprocessed['Overall'])
C:\ProgramData\Anaconda3\envs\Assignment 2\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
<ipython-input-64-caa0ba1998fd>:12: SettingWit

In [65]:
x = X_train.columns
for item in x:
    print(item)
    print(X_train[(item)].dtype)

Age
int64
Nationality
object
Potential
int64
Club
object
Value
object
Wage
float64
Special
int64
Preferred Foot
object
International Reputation
float64
Weak Foot
float64
Skill Moves
float64
Work Rate
object
Body Type
object
Position
object
Jersey Number
float64
Joined
object
Height
float64
Weight
float64
LS
float64
ST
float64
RS
float64
LW
float64
LF
object
CF
float64
RF
float64
RW
float64
LAM
float64
CAM
float64
RAM
float64
LM
float64
LCM
float64
CM
float64
RCM
float64
RM
float64
LWB
float64
LDM
float64
CDM
float64
RDM
float64
RWB
float64
LB
float64
LCB
float64
CB
float64
RCB
float64
RB
float64
Crossing
float64
Finishing
float64
HeadingAccuracy
float64
ShortPassing
float64
Volleys
float64
Dribbling
float64
Curve
float64
FKAccuracy
float64
LongPassing
float64
BallControl
float64
Acceleration
float64
SprintSpeed
float64
Agility
float64
Reactions
float64
Balance
float64
ShotPower
float64
Jumping
float64
Stamina
float64
Strength
float64
LongShots
float64
Aggression
float64
Interceptions
f

In [66]:
test = X_train.LS

In [67]:
test


7151     68.0
9408     60.0
7531     53.0
2057     72.0
6042     61.0
         ... 
6299     60.0
16462    52.0
6556     61.0
1015     78.0
8849     62.0
Name: LS, Length: 9883, dtype: float64

In [41]:
positions = ['LS','ST' , 'RS', 'LW', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM','RCM', 'RM','LWB',
                 'LDM','CDM', 'RDM','RWB', 'LB', 'LCB', 'CB', 'RCB','RB']

In [68]:
for item in positions:
    print(X_train[(item)])


7151     68.0
9408     60.0
7531     53.0
2057     72.0
6042     61.0
         ... 
6299     60.0
16462    52.0
6556     61.0
1015     78.0
8849     62.0
Name: LS, Length: 9883, dtype: float64
7151     68.0
9408     60.0
7531     53.0
2057     72.0
6042     61.0
         ... 
6299     60.0
16462    52.0
6556     61.0
1015     78.0
8849     62.0
Name: ST, Length: 9883, dtype: float64
7151     68.0
9408     60.0
7531     53.0
2057     72.0
6042     61.0
         ... 
6299     60.0
16462    52.0
6556     61.0
1015     78.0
8849     62.0
Name: RS, Length: 9883, dtype: float64
7151     65.0
9408     55.0
7531     54.0
2057     76.0
6042     65.0
         ... 
6299     65.0
16462    56.0
6556     58.0
1015     78.0
8849     66.0
Name: LW, Length: 9883, dtype: float64
7151     67.0
9408     56.0
7531     52.0
2057     75.0
6042     62.0
         ... 
6299     63.0
16462    55.0
6556     59.0
1015     78.0
8849     65.0
Name: CF, Length: 9883, dtype: float64
7151     67.0
9408     56.0
7531   

In [47]:
x = X_train[positions[0]]

In [50]:
item

66+2
58+2
51+2
70+2
59+2
56+2
44+2
57+2
60+2
54+2
69+2
72+2
54+2
72+2
57+2
63+2
59+2
50+2
64+2
38+2
57+2
61+2
64+2
53+2
52+2
60+2
44+2
54+2
35+2
66+2
52+2
69+2
54+2
64+2
57+2
63+2
66+2
64+2
40+2
70+2
72+2
70+2
64+2
61+2
58+2
58+2
51+2
47+2
70+2
62+2
49+2
60+2
60+2
60+2
59+2
62+2
65+2
68+2
69+2
57+2
55+2
54+2
65+2
67+2
62+2
73+3
43+2
63+2
70+2
77+3
68+2
61+2
61+2
72+2
71+2
64+2
62+2
62+2
50+2
60+2
65+2
47+2
63+2
65+2
48+2
58+2
58+2
66+2
61+2
49+2
55+2
72+2
69+2
52+2
62+2
44+2
53+2
59+2
58+2
62+2
52+2
68+2
49+2
55+2
58+2
48+2
63+2
64+2
60+2
71+2
57+2
67+2
49+2
55+2
53+2
71+2
63+2
62+2
63+2
56+2
64+2
43+2
58+2
43+2
59+2
59+2
61+2
55+2
67+2
63+2
59+2
48+2
44+2
67+2
56+2
71+2
51+2
46+2
56+2
56+2
47+2
54+2
39+2
67+2
62+2
54+2
62+2
61+2
44+2
56+2
52+2
58+2
44+2
58+2
65+2
68+2
58+2
58+2
59+2
51+2
50+2
44+2
55+2
56+2
56+2
59+2
72+2
60+2
69+2
60+2
69+2
51+2
46+2
64+2
71+2
69+2
65+2
46+2
58+2
64+2
52+2
53+2
44+2
61+2
62+2
69+2
60+2
62+2
49+2
55+2
60+2
68+2
67+2
50+2
75+2
62+2
65+2
67+2
49+2
48+2
